In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install torch==2.1.0 torchvision==0.16.0 --upgrade --quiet

In [ ]:
!pip install transformers==4.40.0
!pip install av
!pip install imageio
!pip install decord
!pip install opencv-python
!pip install flash-attn --no-build-isolation
!pip install deepspeed

In [ ]:
!pip install --upgrade deepspeed bitsandbytes

!pip install -U huggingface_hub

!pip install peft==0.10.0

!pip install accelerate==0.28.0

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4 --no-cache-dir --force-reinstall


In [ ]:
!python -m bitsandbytes

In [ ]:
!git clone https://github.com/OpenGVLab/InternVL.git

In [ ]:
# !pip install -r /content/InternVL/requirements/internvl_chat.txt

In [ ]:
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer


# model setting
model_path = "OpenGVLab/InternVL2_5-2B" # 'OpenGVLab/InternVideo2_5_Chat_8B'

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda().to(torch.bfloat16)

In [ ]:
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([T.Lambda(lambda img: img.convert("RGB") if img.mode != "RGB" else img), T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC), T.ToTensor(), T.Normalize(mean=MEAN, std=STD)])
    return transform


def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float("inf")
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio


def dynamic_preprocess(image, min_num=1, max_num=6, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set((i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = ((i % (target_width // image_size)) * image_size, (i // (target_width // image_size)) * image_size, ((i % (target_width // image_size)) + 1) * image_size, ((i // (target_width // image_size)) + 1) * image_size)
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images


def load_image(image, input_size=448, max_num=6):
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values


def get_index(bound, fps, max_frame, first_idx=0, num_segments=32):
    if bound:
        start, end = bound[0], bound[1]
    else:
        start, end = -100000, 100000
    start_idx = max(first_idx, round(start * fps))
    end_idx = min(round(end * fps), max_frame)
    seg_size = float(end_idx - start_idx) / num_segments
    frame_indices = np.array([int(start_idx + (seg_size / 2) + np.round(seg_size * idx)) for idx in range(num_segments)])
    return frame_indices

def get_num_frames_by_duration(duration):
        local_num_frames = 4
        num_segments = int(duration // local_num_frames)
        if num_segments == 0:
            num_frames = local_num_frames
        else:
            num_frames = local_num_frames * num_segments

        num_frames = min(512, num_frames)
        num_frames = max(128, num_frames)

        return num_frames

def load_video(video_path, bound=None, input_size=448, max_num=1, num_segments=32, get_frame_by_duration = False):
    vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    max_frame = len(vr) - 1
    fps = float(vr.get_avg_fps())

    pixel_values_list, num_patches_list = [], []
    transform = build_transform(input_size=input_size)
    if get_frame_by_duration:
        duration = max_frame / fps
        num_segments = get_num_frames_by_duration(duration)
    frame_indices = get_index(bound, fps, max_frame, first_idx=0, num_segments=num_segments)
    for frame_index in frame_indices:
        img = Image.fromarray(vr[frame_index].asnumpy()).convert("RGB")
        img = dynamic_preprocess(img, image_size=input_size, use_thumbnail=True, max_num=max_num)
        pixel_values = [transform(tile) for tile in img]
        pixel_values = torch.stack(pixel_values)
        num_patches_list.append(pixel_values.shape[0])
        pixel_values_list.append(pixel_values)
    pixel_values = torch.cat(pixel_values_list)
    return pixel_values, num_patches_list



In [ ]:
# evaluation setting
max_num_frames = 512
generation_config = dict(
    do_sample=False,
    temperature=0.0,
    max_new_tokens=1024,
    top_p=0.1,
    num_beams=1
)
video_path = "/content/drive/MyDrive/deception_detection/train/trial_lie_001_000.mp4"
num_segments=128


with torch.no_grad():
  torch.cuda.empty_cache()
  import gc
  gc.collect()
  pixel_values, num_patches_list = load_video(video_path, num_segments=num_segments, max_num=1, get_frame_by_duration=False)
  print(len(num_patches_list))
  pixel_values = pixel_values.to(torch.bfloat16).to(model.device)
  video_prefix = "".join([f"Frame{i+1}: <image>\n" for i in range(len(num_patches_list))])
  # single-turn conversation
  question1 = "Describe this video in detail."
  question = video_prefix + question1
  output1, chat_history = model.chat(tokenizer, pixel_values, question, generation_config, num_patches_list=num_patches_list, history=None, return_history=True)
  print(output1)
  del output1
  torch.cuda.empty_cache()
  import gc
  gc.collect()
  # # multi-turn conversation
  question2 = "How many people appear in the video?"
  output2, chat_history = model.chat(tokenizer, pixel_values, question, generation_config, num_patches_list=num_patches_list, history=chat_history, return_history=True)

  print(output2)


### Training

In [ ]:
import os
import json
from typing import List, Dict
import random


def build_internvl_video_jsonl(
    video_dir: str,
    output_jsonl_path: str,
    prompt_text: str = "Predict if the person in this video is lying (output \"lie\") or telling the truth (output \"truth\")."
) -> None:
    """
    Converts a folder of video clips into the InternVL chat data format (JSONL).

    Args:
        video_dir (str): Directory containing .mp4 video files (named with 'lie' or 'truth' in the name).
        output_jsonl_path (str): File path to save the resulting JSONL dataset.
        prompt_text (str): Text prompt for the human in each conversation.
    """
    data_entries: List[Dict] = []
    video_filenames = sorted(os.listdir(video_dir))
    video_id = 0

    for fname in video_filenames:
        if not fname.endswith(".mp4"):
            continue

        parts = fname.split("_")
        if len(parts) < 3:
            print(f"Skipping malformed filename: {fname}")
            continue

        label_str = parts[1].lower()
        if label_str not in ["lie", "truth"]:
            print(f"Unknown label in filename: {fname}")
            continue

        label = label_str  # 'lie' or 'truth'
        video_path = fname  # relative path for InternVL

        entry = {
            "id": video_id,
            "video": video_path,
            "conversations": [
                {
                    "from": "human",
                    "value": f"<video>\n{prompt_text}"
                },
                {
                    "from": "gpt",
                    "value": label
                }
            ]
        }

        data_entries.append(entry)
        video_id += 1

    random.shuffle(data_entries)
    # Save to JSONL
    with open(output_jsonl_path, "w", encoding="utf-8") as f:
        for entry in data_entries:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"Saved {len(data_entries)} entries to {output_jsonl_path}")


# Example usage (adjust paths in Colab!)
ROOT_DIR = "/content/drive/MyDrive/deception_detection/train"
OUTPUT_JSONL = "/content/drive/MyDrive/deception_detection/internvl_train.jsonl"

build_internvl_video_jsonl(
    video_dir=ROOT_DIR,
    output_jsonl_path=OUTPUT_JSONL
)


In [ ]:
import os

meta = {
    "deception_train_set": {
        "root": "/content/drive/MyDrive/deception_detection/train/",
        "annotation": "/content/drive/MyDrive/deception_detection/internvl_train.jsonl",
        "data_augment": False,
        "max_dynamic_patch": 4,
        "repeat_time": 1,
        "length": 500  # <- replace with actual count if known
    }
}
os.makedirs("/content/drive/MyDrive/deception_detection", exist_ok=True)

meta_path = "/content/drive/MyDrive/deception_detection/meta_train.json"
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2)

print("✅ Meta file created:", meta_path)


In [ ]:
import os
from huggingface_hub import HfApi, HfFolder, snapshot_download, create_repo, upload_folder

# Parameters
GPUS = 1
BATCH_SIZE = 8
PER_DEVICE_BATCH_SIZE = 2
GRADIENT_ACC = BATCH_SIZE // PER_DEVICE_BATCH_SIZE // GPUS
MASTER_PORT = 34229

# Save to Google Drive
HF_MODEL_NAME = "InternVL2_5-2B-deception-finetuned"  # "internvideo2_5-8b-deception-finetuned"
DRIVE_OUTPUT_DIR = f"/content/drive/MyDrive/deception_detection/{HF_MODEL_NAME}"
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)

# Also save locally for training compatibility
LOCAL_OUTPUT_DIR = f"work_dirs/internvl_chat_v2_5/{HF_MODEL_NAME}"
os.makedirs(LOCAL_OUTPUT_DIR, exist_ok=True)

# Export environment
os.environ['PYTHONPATH'] = f"{os.environ.get('PYTHONPATH', '')}:{os.getcwd()}"
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#os.environ['MASTER_PORT'] = str(MASTER_PORT)
os.environ["PYTHONPATH"] += ":/content/InternVL/internvl_chat"
os.environ["LAUNCHER"] = "pytorch"

!export LD_LIBRARY_PATH=/usr/lib64-nvidia:$LD_LIBRARY_PATH

os.environ["LD_LIBRARY_PATH"] = "/usr/lib64-nvidia:" + os.environ.get("LD_LIBRARY_PATH", "")


In [ ]:
!nvidia-smi

In [ ]:
!pip install decord==0.6.0 --no-cache-dir


In [ ]:
CUSTOM_META_PATH = "/content/drive/MyDrive/deception_detection/meta_train.json"
TRAIN_SCRIPT_PATH = "InternVL/internvl_chat/internvl/train/internvl_chat_finetune.py"
DEEPSPEED_CONFIG = "InternVL/internvl_chat/zero_stage1_config.json"

!CUDA_LAUNCH_BLOCKING=1 TORCH_DISTRIBUTED_DEBUG=DETAIL \
torchrun --nproc_per_node=1 \
  "InternVL/internvl_chat/internvl/train/internvl_chat_finetune.py" \
  --model_name_or_path "OpenGVLab/InternVL2_5-2B" \
  --conv_style "internvl2_5" \
  --use_fast_tokenizer False \
  --output_dir "/content/drive/MyDrive/deception_detection/internvl2.5-2b-deception-finetuned" \
  --meta_path "/content/drive/MyDrive/deception_detection/meta_train.json" \
  --overwrite_output_dir True \
  --force_image_size 224 \
  --max_dynamic_patch 4 \
  --down_sample_ratio 0.5 \
  --drop_path_rate 0.0 \
  --freeze_llm True \
  --freeze_mlp True \
  --freeze_backbone True \
  --use_llm_lora 16 \
  --vision_select_layer -1 \
  --dataloader_num_workers 2 \
  --fp16 True \
  --num_train_epochs 1 \
  --per_device_train_batch_size 1 \
  --gradient_accumulation_steps 4 \
  --evaluation_strategy "no" \
  --save_strategy "steps" \
  --save_steps 200 \
  --save_total_limit 1 \
  --learning_rate 4e-5 \
  --weight_decay 0.05 \
  --warmup_ratio 0.03 \
  --lr_scheduler_type "cosine" \
  --logging_steps 1 \
  --max_seq_length 8192 \
  --do_train True \
  --grad_checkpoint True \
  --group_by_length True \
  --dynamic_image_size True \
  --use_thumbnail True \
  --ps_version "v2" \
  --deepspeed "InternVL/internvl_chat/zero_stage1_config.json" \
  --report_to "tensorboard"